# Video 4: Lazy expressions using a notebook

#### Step 0: Installation and imports

In [ ]:
!pip install caterva2 blosc2 blosc2_grok matplotlib

In [ ]:
from time import time

import blosc2
import matplotlib.pyplot as plt
import numpy as np

import caterva2 as cat2

url = "https://cat2.cloud/demo"

#### Step 1: Setup client and get root

In [ ]:
client = cat2.Client(url, ("user@example.com", "foobar11"))
client.get_roots()
myroot = client.get("@personal")
print(myroot)
print(myroot.file_list)

#### Step 2: Create and upload the arrays to the server


In [ ]:
N = 1_000
dtype = "float64"

# Create and upload the arrays to the server
t0 = time()
a = blosc2.linspace(0, 1, N * N, dtype=dtype, shape=(N, N), urlpath="a.b2nd", mode="w")
b = blosc2.linspace(1, 2, N * N, dtype=dtype, shape=(N, N), urlpath="b.b2nd", mode="w")
c = blosc2.linspace(0, 1, N, dtype=dtype, shape=(N,), urlpath="c.b2nd", mode="w")

ra, rb, rc = (
    client.upload("a.b2nd", "@personal/a.b2nd"),
    client.upload("b.b2nd", "@personal/b.b2nd"),
    client.upload("c.b2nd", "@personal/c.b2nd"),
)
tarray = time() - t0
fs = a.schunk.nbytes + b.schunk.nbytes + c.schunk.nbytes
print(
    f"Time for generating and uploading data (compressed): {tarray:.3f}s"
    f" - data size: {fs / 2 ** 20:.2f} MB"
)

In [ ]:
le = client.upload(ra + 1, "@personal/le2.b2nd")

#### Step 3: Send lazy expressions to server

In [ ]:
rle = client.upload(blosc2.sin(ra) + rc, "@personal/le.b2nd")
# Compute a slice of the remote array
t0 = time()
result = rle.slice(slice(5, 9))
t1 = time() - t0
fs = result.schunk.nbytes
print(
    f"Time for computing and downloading data (compressed): {t1:.3f}s" f" - data size: {fs / 2 ** 10:.2f} KB"
)

# Compute a slice of the remote array
t0 = time()
result = rle[5:9]
t1 = time() - t0
print(
    f"Time for computing and downloading data (uncompressed): {t1:.3f}s"
    f" - data size: {fs / 2 ** 10:.2f} KB"
)

np.allclose((blosc2.sin(a) + c)[5:9], result[:], rtol=1e-6)

##### A more complicated expression

In [ ]:
lexpr = blosc2.lazyexpr("sum(((ra**3 + sin(ra**2)) < rc) & (rb > 0), axis = 1)")
rle = client.upload(lexpr, "@personal/le.b2nd")
# Compute a slice of the remote array
t0 = time()
result = rle.slice(slice(5, 9))
t1 = time() - t0
fs = result.schunk.nbytes
print(
    f"Time for computing and downloading data (compressed): {t1:.3f}s" f" - data size: {fs / 2 ** 10:.2f} KB"
)

In [ ]:
result[:]

## Bonus: Remote data processing with cat2cloud

In [ ]:
myroot = client.get("@public")
myfile = "lung-jpeg2000_10x.b2nd"
address = "examples/" + myfile
ds = myroot[address]
print(ds)
print(f"Dataset shape is {ds.shape} and dtype is {ds.dtype}")

# Download and decompress slice of dataset
plt.imshow(ds[5][0])
plt.gca().axis("off");

##### Method 1: Download data and compute locally

In [ ]:
# Download compressed data and compute std and mean
slice_ = np.index_exp[:, :, :]
t0 = time()
data = ds.slice(slice_)
s = blosc2.std(data, axis=0)
m = blosc2.mean(data, axis=0)
t1 = time() - t0
print(f"std_centre = {s[624, 1345]}, mean_centre = {m[624, 1345]}")
print(f"std_offcentre = {s[20, 20]}, mean_offcentre = {m[20, 20]}")
print(f"Total time = {round(t1, 4)} s")

##### Method 2: Upload lazy expression, compute remotely and download result


In [ ]:
# Send instructions to server and download std and mean
t0 = time()
le_std = blosc2.lazyexpr("std(ds, axis=0)")
le_mean = blosc2.lazyexpr("mean(ds, axis=0)")
s = client.upload(le_std, "@personal/le_std.b2nd").slice(slice_)
m = client.upload(le_mean, "@personal/le_mean.b2nd").slice(slice_)
t1 = time() - t0
print(f"std_centre = {s[624, 1345]}, mean_centre = {m[624, 1345]}")
print(f"std_offcentre = {s[20, 20]}, mean_offcentre = {m[20, 20]}")
print(f"Total time = {round(t1, 4)} s")

To execute this notebook in a browser running on WSL, run ``jupyter notebook --port=8889 --no-browser`` in the WSL terminal.